# LeNet 5 Model Architecture

In [2]:
#importing necessary libralies
import keras
from keras.datasets import mnist
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten
from keras.models import Sequential

#Loading the dataset and perform splitting
(x_train, y_train),(x_test, y_test) = mnist.load_data()

#Performing reshaping operation
x_train = x_train.reshape(x_train.shape[0],28,28,1)
x_test = x_test.reshape(x_test.shape[0],28,28,1)

#Normalization
x_train = x_train/255
x_test = x_test/255

#One Hot Encoding
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

#Building the Model Architecture
model = Sequential()
model.add(Conv2D(6,kernel_size=(5,5),activation='relu',input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(16, kernel_size=(5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(120, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss=keras.metrics.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1, validation_data=(x_test,y_test))

score = model.evaluate(x_test, y_test)
('Test Loss:', score[0])
('Test accuracy:', score[1])

Epoch 1/20
469/469 [==============================] - 7s 15ms/step - loss: 0.3114 - accuracy: 0.9071 - val_loss: 0.0980 - val_accuracy: 0.9708
Epoch 2/20
469/469 [==============================] - 10s 21ms/step - loss: 0.0924 - accuracy: 0.9718 - val_loss: 0.0677 - val_accuracy: 0.9773
Epoch 3/20
469/469 [==============================] - 8s 17ms/step - loss: 0.0658 - accuracy: 0.9800 - val_loss: 0.0551 - val_accuracy: 0.9829
Epoch 4/20
469/469 [==============================] - 9s 19ms/step - loss: 0.0514 - accuracy: 0.9849 - val_loss: 0.0492 - val_accuracy: 0.9845
Epoch 5/20
469/469 [==============================] - 7s 15ms/step - loss: 0.0444 - accuracy: 0.9863 - val_loss: 0.0389 - val_accuracy: 0.9868
Epoch 6/20
469/469 [==============================] - 9s 20ms/step - loss: 0.0371 - accuracy: 0.9883 - val_loss: 0.0370 - val_accuracy: 0.9873
Epoch 7/20
469/469 [==============================] - 8s 17ms/step - loss: 0.0332 - accuracy: 0.9898 - val_loss: 0.0360 - val_accuracy: 0.989

('Test accuracy:', 0.9901000261306763)